In [1]:
!pip install --upgrade --force-reinstall --no-cache-dir ../../../

Processing /home/sbumseo/git/ivete
  Created wheel for ivete: filename=ivete-1.0.0-py3-none-any.whl size=2272 sha256=9a44f5c892eb48eb15233b6fb12d10d0e45e856e1de8cd6f29da0b76d268b64b
  Stored in directory: /tmp/pip-ephem-wheel-cache-a8xg9ta8/wheels/d3/e8/99/b45fc6158bfdebc264a05d8781941b49d19db5b0694e57ecb4
Successfully built ivete
  Attempting uninstall: ivete
    Found existing installation: ivete 1.0.0
    Uninstalling ivete-1.0.0:
      Successfully uninstalled ivete-1.0.0


In [2]:
%pwd

'/home/sbumseo/git/ivete/ivete/experiments/qa'

In [12]:
import torch
from transformers import BertConfig, BertForQuestionAnswering, BertTokenizer
from ivete.deploy import DeployArguments

In [14]:
args = DeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_path="./checkpoints/",
    inference_args={
        'max_seq_length': 128,
        'max_query_length': 32,
    }
)

TypeError: __init__() got an unexpected keyword argument 'inference_args'

In [8]:
checkpoint = torch.load(
    args.downstream_model_path,
    map_location=torch.device("cpu"),
)

In [9]:
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name
)
model = BertForQuestionAnswering(pretrained_model_config)

In [10]:
model.load_state_dict({k.replace("model.", ""): v for k, v in checkpoint['state_dict'].items()})

<All keys matched successfully>

In [11]:
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [13]:
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

In [ ]:
def inference(question, context):
    if question and context:
        truncated_query = tokenizer.encode(
            question,
            add_special_tokens=False,
            truncation=True,
            max_length=args.inference_args['max_query_length'],
       )
        inputs = tokenizer.encode_plus(
            text=truncated_query,
            text_pair=context,
            truncation="only_second",
            padding="max_length",
            max_length=args.inference_args['max_seq_length'],
            return_token_type_ids=True,
        )
        with torch.no_grad():
            outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})
            start_pred = outputs.start_logits.argmax(dim=-1).item()
            end_pred = outputs.end_logits.argmax(dim=-1).item()
            pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1])
    else:
        pred_text = ""
    return {
        'question': question,
        'context': context,
        'answer': pred_text,
    }